In [ ]:
import pandas as pd
import re
from snownlp import SnowNLP

with open ("逆袭_merged.txt","r",encoding="utf-8") as f:
    comments=[line.strip() for line in f.readlines()]

df=pd.DataFrame(comments,columns=["raw_comment"])

df=df.drop_duplicates(subset=["raw_comment"],keep="first")
df=df[df["raw_comment"] !=""]
df=df.reset_index(drop=True)

def clean_text(text):
     emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  
        "\U0001F300-\U0001F5FF" 
        "\U0001F680-\U0001F6FF"  
        "\U0001F1E0-\U0001F1FF"  
        "\U00002702-\U000027B0"  
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
     text+emoji_pattern.sub(r"",text)

     text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9，。！？,!.?]", " ", text)
     text = re.sub(r"\s+", " ", text).strip()

     return text

df["cleaned_comment"]=df["raw_comment"].apply(clean_text)

df = df[df["cleaned_comment"] != ""]
df = df.reset_index(drop=True)


print("原始评论示例：")
print(df["raw_comment"].head(3).tolist())
print("\n清洗后评论示例：")
print(df["cleaned_comment"].head(3).tolist())
print(f"\n处理后剩余有效评论数：{len(df)}")

原始评论示例：
['[赞][赞][赞]这个短剧说的太现实了，人老了，一定要给自己留养老钱[赞][赞][赞][赞][赞][赞][赞][鼓掌][鼓掌][鼓掌][鼓掌][鼓掌][鼓掌][鼓掌][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰]很好看的短剧', '[666][666][666]', '[一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油][一起加油]每天坚持 多看广告，关注、点赞，多评论，收藏、转发,，在评论下多回复，为了碎银几两，每天都在看广告，硬是把广告看成了最爱，加油加油加油加油[赞][赞][赞][赞][赞][赞][赞][赞][赞][赞][赞][爱心][爱心][爱心][爱心][爱心][爱心][爱心][爱心][爱心][爱心][感谢][感谢][感谢][感谢][感谢][感谢][感谢][感谢][感谢][感谢][感谢][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰][玫瑰]']

清洗后评论示例：
['赞 赞 赞 这个短剧说的太现实了，人老了，一定要给自己留养老钱 赞 赞 赞 赞 赞 赞 赞 鼓掌 鼓掌 鼓掌 鼓掌 鼓掌 鼓掌 鼓掌 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 很好看的短剧', '666 666 666', '一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 每天坚持 多看广告，关注 点赞，多评论，收藏 转发,，在评论下多回复，为了碎银几两，每天都在看广告，硬是把广告看成了最爱，加油加油加油加油 赞 赞 赞 赞 赞 赞 赞 赞 赞 赞 赞 爱心 爱心 爱心 爱心 爱心 爱心 爱心 爱心 爱心 爱心 感谢 感谢 感谢 感谢 感谢 感谢 感谢 感谢 感谢 感谢 感谢 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰 玫瑰']

处理后剩余有效评论数：3009


In [ ]:
def predict_sentiment_snownlp(text):
    s = SnowNLP(text)
    score = s.sentiments  
    if score > 0.6:
        return "正面", 4 + int((score - 0.6) * 2.5)  
    elif score < 0.4:
        return "负面", 1 + int((0.4 - score) * 2.5)  
    else:
        return "中性", 3  

df[["polarity", "intensity"]] = df["cleaned_comment"].apply(
    lambda x: pd.Series(predict_sentiment_snownlp(x))
)

output_path = "逆袭_sentiment.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"结果已保存至：{output_path}")
print(df[["cleaned_comment", "polarity", "intensity"]].head())

结果已保存至：逆袭_sentiment.csv
                                     cleaned_comment polarity  intensity
0  赞 赞 赞 这个短剧说的太现实了，人老了，一定要给自己留养老钱 赞 赞 赞 赞 赞 赞 赞 ...       正面          5
1                                        666 666 666       中性          3
2  一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一起加油 一...       正面          5
3                                                666       中性          3
4  女人拼了命生下来的孩子，老了有钱还能回来看看，没钱就觉得是累赘，图啥，所以女人还是要自己疼自...       负面          1
